In [1]:
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import glob
import os
from skimage.transform import resize
import cv2
import itertools
import warnings
warnings.filterwarnings('ignore')
import skimage as sk

#torch.set_default_tensor_type('torch.FloatTensor')

In [13]:
class ConvLSTMCell(nn.Module):
    def __init__(self, input_size, input_dim, hidden_dim, kernel_size, bias):
        super(ConvLSTMCell, self).__init__()
    
        self.height, self.width = input_size
        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        
        self.padding     = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias        = bias
        
        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)
        
        self.linear = nn.Linear(in_features=self.hidden_dim*self.height*self.width,out_features=1,bias=self.bias)
        
    def forward(self, input_tensor, cur_state):
        
        if cur_state is None:
            cur_state = (Variable(torch.zeros(1, self.hidden_dim, self.height, self.width)),
                Variable(torch.zeros(1, self.hidden_dim, self.height, self.width)))
        
        h_cur, c_cur = cur_state
        
        #print(input_tensor.size(),h_cur.size())
        combined = torch.cat((input_tensor, h_cur), dim=1)  # concatenate along channel axis
        #print(combined.size())
        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1) 
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)
        
        next_state = (h_next,c_next)
        
        sigmoid_out = F.sigmoid(self.linear(h_next.view(1,-1)))
        
        return next_state,sigmoid_out
    

In [14]:

c, h, w = 5, 4, 8
d = 10
#T = 6
model = ConvLSTMCell((h,w),c,d,(3,3),True)

In [20]:
x1 = Variable(torch.rand(4, c, h, w))
x2 = Variable(torch.rand(7, c, h, w))
y1 = Variable(torch.randn(4, d, h, w))
y2 = Variable(torch.randn(7, d, h, w))
real = Variable(torch.from_numpy(np.array([1,0]).reshape(-1,1))).float()
x = [x1,x2]
y = [y1,y2]
real[0].view(-1,1).size()[0]

1

In [16]:
max_epoch = 200000
lr = 1.e-2

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=lr)
for epoch in range(1, max_epoch+1):
    
    for k in range(len(x)):
        inp = x[k]
        #out = y[k]
        state = None
        #loss = 0
        #print(inp.size(),out.size())
        for t in range(0, inp.size()[0]):
            state,sig_out = model.forward(inp[t:t+1,:,:,:],state)
        
        last_hidden = state[0]
        last_sig = sig_out
        #print(last_sig.size())
        #sig = model.sig_out(last_hidden)
        loss = loss_fn(last_sig, real[k].view(-1,1))
        #print(last_sig,real[k])
        if epoch %1000 == 0:
            print(' > Epoch {:2d} loss: {:.5f}'.format((epoch), loss.data[0]))
            #print(last_sig,real[k])
    
        model.zero_grad()

    # compute new grad parameters through time!
        loss.backward()

    # learning_rate step against the gradient
        """for p in model.parameters():
            p.data.sub_(p.grad.data * lr)"""
        optimizer.step()
            #print(p)
    
    #lr *= 1.00001